# Apps Data Project

Analysing the Google Playstore and iOS Appstore to understand the characteristics of successful apps. 

In [1]:
apple_open_file = open('applestore.csv')
from csv import reader
apple_read_file = reader(apple_open_file)
ios_data = list(apple_read_file)

google_open_file = open('googleplaystore.csv')
from csv import reader
google_read_file = reader(google_open_file)
goog_data = list(google_read_file)

Importing the two data sets as 'ios_data' and 'goog_data'.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

Function used to print the first few rows of the data and define how many rows and columns there are in the data set. 

In [3]:
Google_data = explore_data(goog_data, 0, 4, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10842
Number of columns: 13


In [4]:
Apple_data = explore_data(ios_data, 0, 4, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7198
Number of columns: 16


# Data Cleaning

In [9]:
for rating in goog_data[1:]:
    rating_score = float(rating[2])
    if rating_score > 5:
        print(goog_data.index(rating))

del goog_data[10473] #Only run once!
print('Number of rows:', len(goog_data))

10473
Number of rows: 10841


From the comments on the data source there is one app where the rating is 19 (outside the expected range 0 - 5). This code finds the index number of that app and deletes it, confirming there is now one less row. 

In [13]:
duplicate_apps = []
unique_apps = []

for app in goog_data[1:]:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:10])

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


Checking through the google play store data it is clear there are duplicate apps, where several apps have the same name. These will be removed, with the apps with the highest number of reviews being kept. But first need to check the apple store for duplicate values. 

In [14]:
apple_duplicate_apps = []
apple_unique_apps = []

for app in ios_data[1:]:
    name = app[0]
    if name in apple_unique_apps:
        apple_duplicate_apps.append(name)
    else:
        apple_unique_apps.append(name)
        
print('Number of duplicate apps:', len(apple_duplicate_apps))

Number of duplicate apps: 0


In [55]:
expected_length = len(goog_data[1:]) - len(duplicate_apps)
print('Expected length:', expected_length)

reviews_max = {}
for app in goog_data[1:]:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
        
print('Unique app list length:', len(reviews_max))

goog_data_clean = []
already_added = []

for app in goog_data[1:]:
    name = app[0]
    n_reviews = float(app[3])
    if reviews_max[name] == n_reviews and name not in already_added:
        goog_data_clean.append(app)
        already_added.append(name)
        
print('Cleaned app list length:', len(goog_data_clean))

Expected length: 9659
Unique app list length: 9659
Cleaned app list length: 9659


The code above works through the google play store app list to identify unique apps and pull them into a seperate dictionary. It then creates a new list of lists, goog_data_clean, which contains unique apps (pulling the information for the most number of reviews). 

In [48]:
def english_check(a_list):   
    n_not_english = 0
    for value in a_list:
        if ord(value) > 127:
            n_not_english += 1
    if n_not_english >3:
        return False
    else:
        return True

This function checks if more than 3 characters are not standard english characters (therefore indicating the app is non-english). This is an imperfect method but should generate a good enough filter.  

In [58]:
goog_data_clean_eng = []
ios_data_clean_eng = []

for app in goog_data_clean:
    name = app[0]
    if english_check(name):
        goog_data_clean_eng.append(app)
        
for app in ios_data[1:]:
    name = app[1]
    if english_check(name):
        ios_data_clean_eng.append(app)
        
print(explore_data(goog_data_clean_eng, 0, 3, True))
print('\n')
print(explore_data(ios_data_clean_eng, 0, 3, True))
        

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13
None


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '

Through looping our data through the english_check function we have removed any non-english apps from our data set. 


In [64]:
free_goog_data = []
free_ios_data = []

for app in goog_data_clean_eng:
    name = app[0]
    price = app[7]
    if price == '0':
        free_goog_data.append(app)

for app in ios_data_clean_eng:
    name = app[1]
    price = app[4]
    if price == '0.0':
        free_ios_data.append(app)
        
print(len(free_goog_data))
print(len(free_ios_data))

8864
3222


# Analysis

Using the cleaned data set the objective is to identify what types of free apps are successful on the google play store and the apple store. 

In [83]:
def freq_table(dataset, index):
    dictionary = {}
    total = 0
    for app in dataset:
        total += 1
        genre = app[index]
        if genre in dictionary:
            dictionary[genre] += 1
        else:
            dictionary[genre] = 1
    table_percentages = {}
    for key in dictionary:
        percentage = (dictionary[key] / total) * 100
        table_percentages[key] = percentage
    return table_percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

The code above defines two functions, one which calculates a frequency table for a given data set with a given index (column from the data set, which gives a set of categories). The second function displays this table so that it is easy to interpret. 

In [88]:
display_table(free_goog_data, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [90]:
display_table(free_ios_data, 11)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


The frequency of apps on the two stores are surprisingly different, with apple dominated by games while google has a much more balanced porfolio of apps. 